# Creating a DTM

First importing all the standard python libraries.

In [1]:
import textmining
import shorttext
import pandas as pd
import numpy as np
import numpy.random as random
import matplotlib.pyplot as plt

Using TensorFlow backend.


Using the US Presidents speeches dataset to create a Document term matrix.

In [2]:
usprez = shorttext.data.inaugural()
docids=sorted(usprez.keys())
docsall=[]
for key in docids:
    s=''.join(usprez[key])
    docsall.append(s)

Next using the 'TermDocumentMatrix()' of the 'text mining' library to create the DTM.

In [3]:
tdm = textmining.TermDocumentMatrix()
for d in docsall:
    tdm.add_doc(d)
#tdm.write_csv('matrix.csv', cutoff=1)
c=0
ans=[]
for row in tdm.rows(cutoff=1):
    if(c==0):
        c=1
        continue
    ans.append(row)

Using numpy functions to find the dimensions and different features of the matrix.

In [4]:
a=np.array(ans)
print(a.shape)

(56, 8971)


In [5]:
#finding the transpose of the DTM, will be used later in the multiplication part
b=a.transpose()

Now we are multiplying the DTM matrix and its tranpose and calculating the time taken by the operation.

In [6]:
#importing time library for calculating time
import time
time_starting=time.time()
d=np.matmul(b,a)
time_after=time.time()
print(time_after-time_starting)

7.541239261627197


# Matrix Multiplication using sampling

First calculating the square of the Frobenius norm of DTM.

In [7]:
frobenius_norm_square=np.sum(a**2)
frobenius_norm_square

6665205

Finding the probability using length square sampling.

In [8]:
probability_list=[]
sum=0;
for i in range(np.size(b,1)):
    q=np.array(b[:,i])
    probability_list.append((np.sum(q**2))/(1.0*frobenius_norm_square))

Considering only 1 independant trial.

In [9]:
s=1
rows=np.size(b,1)
#index variable gives us a random number(k) which is the column number of b and corresponding row of a.
index=random.choice(rows,s,probability_list)
print(index)

[14]


Multiplication of the kth column of b and kth row of a

In [11]:
#Using the time library for finding the time taken to calculate this approx matrix
p=np.array(probability_list)
t=p[index]
const=1.0/t[0]
b_kth_column=np.matrix(b[:,index])*const
a_kth_row=np.matrix(a[index])
time_start=time.time()
X=np.dot(b_kth_column[:,None],a_kth_row[None,:])
time_end=time.time()
print(X.shape)
print("Time Taken: " +str(time_end-time_start))

(8971, 8971)
Time Taken: 2.1187150478363037


Considering Multiple trials.

In [12]:
#suppose s=50
s=50
rows=np.size(b,1)
all_index=random.choice(rows,s,probability_list)

In [13]:
#Creating a zero matrix
X=np.zeros((np.size(b,0),np.size(b,0)))

In [15]:
#though the multiple trials method takes a lot more time than the single trials method but it is more accurate
for i in range(0,s):
    t=p[all_index[i]]
    a1=np.array(b[:,all_index[i]])*(1.0/t)
    b1=np.array(a[all_index[i]])
    y=np.dot(a1[:,None],b1[None,:])
    X=np.add(X,y)
X=X*(1.0/s)
print(X.shape)

(8971, 8971)
